In [ ]:
!curl ipecho.net/plain

35.196.65.56

1. selenium(동적 페이지 크롤링을 위함), oepnpyxl(엑셀 저장을 위함) 설치

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install openpyxl

2. 구글드라이브에 연결

In [ ]:
from google.colab import drive
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

drive.mount('/content/drive')

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


3. pandas를 통해 엑셀서식의 앨범Id 파일을 읽어오고 이를 str 타입으로 변환함(기존에는 int 타입이라 추후 웹 주소 결합과정에서 오류발생)

4. 앨범id를 통해 멜론 앨범 페이지에 접근 => check 박스 부분의 value(곡 id)를 추출하여 저장

In [ ]:
#fileNum 수정하며 진행
fileNum = list(range(180,200))

for x in fileNum:
  df = pd.read_excel('/content/drive/MyDrive/team cat/02.입력 데이터처리 출력 과정/Scraper/chunk/file_{:02d}.xlsx'.format(x))
  df1 = df.astype(str)


  SONG_ID=[]
  Not_exist=[]

  for i in df1['album_id']:
    driver.get("https://www.melon.com/album/detail.htm?albumId="+i)
    time.sleep(3)
    try:
      song_id=driver.find_elements(by=By.NAME, value="input_check")  
      for j in song_id:
        SONG_ID.append(j.get_attribute("value"))
    except:
        Not_exist.append(i)
        print(i+"번 앨범이 존재하지 않음")

  df=pd.DataFrame({"song_id":SONG_ID})
  df.to_excel("/content/drive/MyDrive/team cat/02.입력 데이터처리 출력 과정/Scraper/data/songId/songId_{:02d}.xlsx".format(x),  encoding='utf-8')

  df2=pd.DataFrame({"album_id":Not_exist})
  df2.to_excel("/content/drive/MyDrive/team cat/02.입력 데이터처리 출력 과정/Scraper/data/not_exist/ne_album_{:02d}.xlsx".format(x),  encoding='utf-8')

  print(str(x)+"번째 파일 작성 완료")